<a href="https://colab.research.google.com/github/Lakshmi-Adhikari-AI/LLM-HuggingFace/blob/main/course/en/chapter2/section2_pt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Behind the pipeline (PyTorch)

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [32]:
!pip install datasets evaluate transformers[sentencepiece]

In [54]:
#  Pipeline approach - handles preprocessing, model inference, and postprocessing
from transformers import pipeline

# Sentiment analysis pipeline using default pretrained model
classifier = pipeline("sentiment-analysis")

# Predict sentiment for two input sentences
classifier(
    [
        "I love AI",
        "But it's complex so don't want to learn"
    ]
)


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


[{'label': 'POSITIVE', 'score': 0.9998302459716797},
 {'label': 'NEGATIVE', 'score': 0.9895736575126648}]

In [8]:
#  AutoTokenizer - automatically selects correct tokenizer for the given model checkpoint
from transformers import AutoTokenizer

# Raw input sentences to be passed through the model
raw_inputs = [
    "I love AI",
    "But it's complex so don't want to learn"
]

# Tokenizing the raw inputs: adds [CLS], [SEP], and converts to token IDs
# padding=True → ensures all inputs are same length
# truncation=True → trims tokens if they exceed model's max length
# return_tensors="pt" → returns PyTorch tensors
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print(inputs)


In [55]:
#  AutoModel - loads only the base Transformer model (no classification head)
from transformers import AutoModel

# Load pretrained model checkpoint (only base transformer, not task-specific head)
model = AutoModel.from_pretrained(checkpoint)

# Run model on inputs → returns last_hidden_state (token embeddings)
outputs = model(**inputs)
print(outputs.last_hidden_state)
print(outputs.last_hidden_state.shape)  # e.g., torch.Size([2, 16, 768]) → [batch_size, seq_length, hidden_size]


tensor([[[ 5.3020e-01,  2.4128e-01,  6.9855e-02,  ...,  4.2687e-01,
           1.0070e+00, -4.3037e-01],
         [ 6.6918e-01,  4.0507e-01,  6.9381e-02,  ...,  4.8217e-01,
           9.4859e-01, -2.7963e-01],
         [ 9.1037e-01,  4.6725e-01,  3.0641e-01,  ...,  3.7685e-01,
           9.2645e-01, -2.9975e-01],
         ...,
         [ 2.2623e-01,  2.0512e-01, -6.8777e-02,  ...,  4.6769e-01,
           9.5997e-01, -3.5636e-01],
         [ 2.0195e-01,  2.8293e-01, -3.0646e-04,  ...,  4.7270e-01,
           8.5636e-01, -3.9502e-01],
         [ 1.8929e-01,  1.6243e-01, -8.3447e-02,  ...,  5.0290e-01,
           9.6060e-01, -3.6688e-01]],

        [[-1.2220e-01,  5.3712e-01,  1.6431e-02,  ..., -2.6850e-01,
           2.0008e-01,  1.7445e-01],
         [-5.3642e-02,  3.3243e-01, -2.8873e-02,  ..., -2.8136e-01,
           3.1022e-01,  5.7603e-01],
         [-3.5764e-01,  6.1211e-01, -1.7822e-02,  ..., -5.3864e-01,
           1.4982e-01,  2.2937e-01],
         ...,
         [-9.8159e-02,  4

In [57]:
#  AutoModelForSequenceClassification - includes classification head on top of base model
from transformers import AutoModelForSequenceClassification

# Load model with classification head for sentiment analysis task
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

# Run model to get raw predictions (logits)
outputs = model(**inputs)
print(outputs.logits.shape)  # e.g., torch.Size([2, 2]) → 2 sentences, 2 sentiment classes (NEGATIVE, POSITIVE)
print(outputs.logits)        # Raw scores before softmax


torch.Size([2, 2])
tensor([[-4.1860,  4.4950],
        [ 1.3378, -1.1082]], grad_fn=<AddmmBackward0>)


In [58]:
#  Convert logits (raw scores) to probabilities using softmax
import torch
import torch.nn.functional as F

predictions = F.softmax(outputs.logits, dim=1)
print(predictions)  # Each row sums to 1 → [NEGATIVE_prob, POSITIVE_prob]
print(torch.round(predictions * 100) / 100)  # Optional: Round to 2 decimal places for easier reading


tensor([[1.6975e-04, 9.9983e-01],
        [9.2027e-01, 7.9729e-02]], grad_fn=<SoftmaxBackward0>)
tensor([[0.0000, 1.0000],
        [0.9200, 0.0800]], grad_fn=<DivBackward0>)


In [59]:
#  Extract predicted label indices
highest_indices = predictions.argmax(dim=1)  # Returns index of max probability (0 or 1)
print(highest_indices)

#  Map predicted indices to actual label names using model config
labels = [model.config.id2label[idx.item()] for idx in highest_indices]
print(labels)

#  Print results in human-readable format
for i, label in enumerate(labels):
    print(f"Sentence {i+1}: {label}")



tensor([1, 0])
['POSITIVE', 'NEGATIVE']
Sentence 1: POSITIVE
Sentence 2: NEGATIVE
